In [1]:
library(zellkonverter)         # writeH5AD
library(SingleCellExperiment)
library(Matrix)

library(Seurat)   

Registered S3 method overwritten by 'zellkonverter':
  method                                             from      
  py_to_r.pandas.core.arrays.categorical.Categorical reticulate

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,


In [10]:
obj

$N3
An object of class Seurat 
36601 features across 9746 samples within 1 assay 
Active assay: Spatial (36601 features, 0 variable features)
 1 layer present: counts
 1 spatial field of view present: slice1

$N4
An object of class Seurat 
36601 features across 8909 samples within 1 assay 
Active assay: Spatial (36601 features, 0 variable features)
 1 layer present: counts
 1 spatial field of view present: slice1

$N5
An object of class Seurat 
36601 features across 9319 samples within 1 assay 
Active assay: Spatial (36601 features, 0 variable features)
 1 layer present: counts
 1 spatial field of view present: slice1

$N6
An object of class Seurat 
36601 features across 9837 samples within 1 assay 
Active assay: Spatial (36601 features, 0 variable features)
 1 layer present: counts
 1 spatial field of view present: slice1

$N7
An object of class Seurat 
36601 features across 9707 samples within 1 assay 
Active assay: Spatial (36601 features, 0 variable features)
 1 layer present: coun

In [9]:
# ---- R EXPORT ----
# install.packages(c("Matrix","Seurat"))
library(Seurat)
library(Matrix)

in_rds  <- "/Users/christoffer/work/karolinska/development/MitoMetaoMicS/data/dbit_MS/raw_seu_n3_n10.rds"
out_dir <- "/Users/christoffer/work/karolinska/development/MitoMetaoMicS/data/dbit_MS/export_raw_seu_n3_n10"
dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)

obj <- readRDS(in_rds)
assay <- DefaultAssay(obj)

# helper for Seurat v5 layers with fallback
has_layer <- function(a, nm) tryCatch(nm %in% Layers(obj[[a]]), error=function(e) FALSE)
get_layer <- function(a, nm) {
  if (has_layer(a, nm)) {
    GetAssayData(obj, assay=a, layer=nm)
  } else {
    # fallback to slot names for older objects
    slotnm <- switch(nm, counts="counts", data="data", "scale.data"="scale.data", nm)
    tryCatch(GetAssayData(obj[[a]], slot=slotnm), error=function(e) NULL)
  }
}

# 1) counts (dgCMatrix)
counts <- get_layer(assay, "counts")
if (is.null(counts)) stop("No counts matrix found in assay: ", assay)
counts <- as(counts, "dgCMatrix")

# canonical order
cells <- colnames(counts)
genes <- rownames(counts)

# 2) logcounts (use Seurat "data" if present, else compute TP10k+log1p)
logcounts <- get_layer(assay, "data")
if (is.null(logcounts) || nrow(logcounts)==0 || ncol(logcounts)==0) {
  cs <- Matrix::colSums(counts); cs[cs==0] <- 1
  norm <- t(t(counts) / cs) * 1e4
  logcounts <- as(as.matrix(log1p(norm)), "dgCMatrix")
} else {
  # ensure same gene/cell order as counts
  logcounts <- logcounts[genes, cells, drop=FALSE]
  if (!inherits(logcounts, "dgCMatrix")) logcounts <- as(as.matrix(logcounts), "dgCMatrix")
}

# 3) obs (cell metadata)
obs <- obj@meta.data
# align & de-factor
obs <- obs[cells, , drop=FALSE]
obs[] <- lapply(obs, function(x) if (is.factor(x)) as.character(x) else x)
obs$cell_id <- rownames(obs)

# 4) var (gene table)
var <- data.frame(gene=genes, row.names=genes, stringsAsFactors=FALSE)

# 5) embeddings (obsm)
write_emb <- function(red_name, fname){
  if (!red_name %in% names(Reductions(obj))) return(FALSE)
  emb <- Embeddings(obj, red_name)
  emb <- emb[cells, , drop=FALSE]  # align
  write.csv(emb, file=file.path(out_dir, fname), row.names=TRUE)
  TRUE
}
has_pca  <- write_emb("pca",  "X_pca.csv")
has_umap <- write_emb("umap", "X_umap.csv")
has_tsne <- write_emb("tsne", "X_tsne.csv")

# 6) spatial (optional; if present in obj@images)
if (length(Images(obj)) > 0) {
  for (im in Images(obj)) {
    coords <- GetTissueCoordinates(obj, image=im)
    coords <- coords[intersect(rownames(coords), cells), , drop=FALSE]
    write.csv(coords, file=file.path(out_dir, paste0("spatial_", im, ".csv")), row.names=TRUE)
  }
}

# ---- write matrices (Matrix Market) + tsvs ----
Matrix::writeMM(counts, file.path(out_dir, "counts.mtx"))
Matrix::writeMM(logcounts, file.path(out_dir, "logcounts.mtx"))
write.table(genes,  file.path(out_dir, "genes.tsv"),   quote=FALSE, sep="\t", row.names=FALSE, col.names=FALSE)
write.table(cells,  file.path(out_dir, "barcodes.tsv"),quote=FALSE, sep="\t", row.names=FALSE, col.names=FALSE)
write.csv(obs, file=file.path(out_dir, "obs.csv"), row.names=FALSE)
write.csv(var, file=file.path(out_dir, "var.csv"), row.names=TRUE)

cat("Exported to: ", out_dir, "\n")

ERROR: Error in UseMethod(generic = "DefaultAssay", object = object): no applicable method for 'DefaultAssay' applied to an object of class "list"
